In [6]:
# !pip install gdown


In [7]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , LSTM, Dense , Dropout
from tensorflow.keras.initializers import Constant

from sklearn.metrics import confusion_matrix , classification_report
from tensorflow.keras.optimizers import SGD , Adam
import bz2
import csv
from sklearn.metrics import roc_auc_score
import gdown


In [8]:
file_id_train = '1iOcVRDLVhbvenxI3ij6B3gBJXbkTcUee'
url = f'https://drive.google.com/uc?id={file_id_train}'

#https://docs.google.com/spreadsheets/d/1iOcVRDLVhbvenxI3ij6B3gBJXbkTcUee/edit?usp=sharing&ouid=111164914988840298826&rtpof=true&sd=true
# Specify the output file name
output1 = 'your_data1.csv'

# Download the file from Google Drive
gdown.download(url, output1, quiet=False)

# Read the CSV file into a Pandas DataFrame
train_data = pd.read_excel(output1, header=None)

# Display the first few rows of the DataFrame
#train_data.head()


# https://drive.google.com/file/d/1uulB7PXy3v6wWpUGQgdCYogOG_KBiChu/view?usp=drive_link
#https://docs.google.com/spreadsheets/d/1UkVKHzLRnFcWHPuw-egzeoSVWqHRHv9d/edit?usp=drive_link&ouid=111164914988840298826&rtpof=true&sd=true

file_id_test = '1UkVKHzLRnFcWHPuw-egzeoSVWqHRHv9d'
url2 = f'https://drive.google.com/uc?id={file_id_test}'

# Specify the output file name
output2 = 'your_data2.csv'

# Download the file from Google Drive
gdown.download(url2, output2, quiet=False)

# Read the CSV file into a Pandas DataFrame
test_data = pd.read_excel(output2, header=None)

# Display the first few rows of the DataFrame
test_data.head()


# train_data = pd.read_csv("./dataset/train.csv",header = None)

# # train_data.head()

# test_data = pd.read_csv("./dataset/test.csv",header = None)
# # test_data.head()

Downloading...
From: https://drive.google.com/uc?id=1iOcVRDLVhbvenxI3ij6B3gBJXbkTcUee
To: /content/your_data1.csv
100%|██████████| 2.19M/2.19M [00:00<00:00, 94.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UkVKHzLRnFcWHPuw-egzeoSVWqHRHv9d
To: /content/your_data2.csv
100%|██████████| 228k/228k [00:00<00:00, 57.8MB/s]


,0,1,2
0,Column1,Column2,Column3
1,1,mens ultrasheer,"This model may be ok for sedentary types, but ..."
2,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
3,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
4,2,Oh dear,I was excited to find a book ostensibly about ...


In [9]:
train_sentence  = train_data[2]
train_rating = train_data[0]

test_sentence  = test_data[2]
test_rating = test_data[0]

In [15]:
MAX_LEN = 1000
Vocab_size = 10000
embedding_dim = 100

#tokanization
tokenizer = Tokenizer(num_words=Vocab_size)
tokenizer.fit_on_texts(train_sentence.tolist() + test_sentence.tolist())

In [16]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]","",text)
    text = text.strip()
    return text

train_sentence = train_sentence.apply(clean_text)
test_sentence = test_sentence.apply(clean_text)

In [17]:
# Convert sentences to sequences
train_sequence = tokenizer.texts_to_sequences(train_sentence)
test_sequence = tokenizer.texts_to_sequences(test_sentence)

#pad sequence
train_sequence = pad_sequences(train_sequence, maxlen=MAX_LEN)
test_sequence = pad_sequences(test_sequence, maxlen=MAX_LEN)

In [25]:
if not Vocab_size:  # If vocabulary size not explicitly set
    word_index = tokenizer.word_index
    num_words = min(Vocab_size, len(word_index))  # Truncate if necessary
    embedding_matrix = np.zeros((num_words, 128))  # Adjust embedding dimension
    for word, i in word_index.items():
        if i >= num_words:
            continue
        embedding_matrix[i] = ...  # Fill with word embeddings (e.g., from Word2Vec, GloVe)

# Prepare final output for model training
x_train = train_sequence
y_train = train_rating
x_test = test_sequence
y_test = test_rating

# Print data shapes for verification
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (10000, 1000)
y_train shape: (10000,)
x_test shape: (1001, 1000)
y_test shape: (1001,)


In [19]:



word_index = tokenizer.word_index

# Calculate num_words before using it
num_words = min(Vocab_size, len(word_index))
embedding_matrix = np.zeros((num_words, 100))
# Adjust embedding dimension


# Model architecture
model = Sequential()
model.add(Embedding(input_dim=Vocab_size, output_dim=embedding_dim, embeddings_initializer=Constant(embedding_matrix)))

model.add(LSTM(units=64, return_sequences=True))  # Consider multiple LSTM layers
model.add(LSTM(units=32))
model.add(Dense(1, activation='linear'))  # Adjust activation as needed

model.compile(loss='mean_squared_error', optimizer='adam')

In [20]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
313/313 [==============================] - 360s 1s/step - loss: 0.5908 - val_loss: 0.5927
Epoch 2/5
299/313 [===========================>..] - ETA: 15s - loss: 0.3945

In [29]:
from tensorflow.keras import backend as K


#Evaluate performance
loss= model.evaluate(x_val, y_val)
mse = K.mean(K.square(model.output - y_val))
print("Mean Squared Error:", mse)

32/32 [==============================] - 8s 254ms/step - loss: 1.2028
Mean Squared Error: KerasTensor(type_spec=TensorSpec(shape=(), dtype=tf.float32, name=None), name='tf.math.reduce_mean/Mean:0', description="created by layer 'tf.math.reduce_mean'")


In [34]:
# Make predictions on new sentences
new_sentences = ["This is an amazing movie!", "I hated this book. It was terrible."]
new_sequences = tokenizer.texts_to_sequences(new_sentences)
new_sequences = pad_sequences(new_sequences, maxlen=MAX_LEN)
predictions = model.predict(new_sequences)
print("Predicted ratings:", predictions)

1/1 [==============================] - 0s 230ms/step
Predicted ratings: [[3.5844321]
 [1.6525369]]
